# Projet Multimédia: Segementation audio

### Becko Camara; Guobao LI; Rongbo Liu; Shiting LI

But: 
      Rconnaissance de locuteurs dans une bande audio et détection de changement de locuteurs.


Données: 
               Bande audio

Outils:
          1. pyAudioAnalysis
          2. Jupyter

### 1. Segmentation

#### 1.1. Concepiton

##balabala

#### 1.2. Realisation

Voici le code python:

In [7]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
import numpy as np
import math
import matplotlib.pyplot as plt

### 2. Reconnaissance